Да опитаме различни оптимизатори...

In [1]:
import itertools
import os

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
def articles():
    with open('page_revisions_text', 'rb') as text_file:
        pending_article_data = b''
        while True:
            data = text_file.read(1024 ** 2)
            if len(data) == 0:
                break

            articles = data.split(b'\0')
            articles[0] = pending_article_data + articles[0]
            for index, article in enumerate(articles):
                if index + 1 == len(articles):
                    pending_article_data = article
                else:
                    yield article

        print(pending_article_data)
        if len(pending_article_data) != 0:
            yield pending_article_data

In [3]:
subword_text_encoder = tfds.features.text.SubwordTextEncoder.load_from_file('vocab_4096')

In [4]:
BATCH_SIZE = 256
BATCHED_ITEM_LENGTH = 256
BUFFER_SIZE = 1024
TYPE=np.int16

def articles_generator():
    for index, article in enumerate(itertools.islice(articles(), 0, 10000)):
        yield np.array(subword_text_encoder.encode(article + b'\0'), dtype=TYPE)

    # Pad the article count to the batch size
    # We do this to ensure that no data is dropped
    index += 1
    while index % BATCH_SIZE != 0:
        yield np.array([0], dtype=TYPE)
        index += 1

def subbatches():
    dataset = tf.data.Dataset.from_generator(articles_generator, output_types=TYPE)
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.padded_batch(BATCH_SIZE, padded_shapes=([None]), drop_remainder=True)
    dataset = dataset.shuffle(2000)

    for batch in dataset.as_numpy_iterator():
        remaining = batch
        while remaining.shape[1] > 1:
            yield remaining[:, :BATCHED_ITEM_LENGTH + 1]
            remaining = remaining[:, BATCHED_ITEM_LENGTH:]

dataset = tf.data.Dataset.from_generator(subbatches, output_types=TYPE, output_shapes=(BATCH_SIZE, None))
dataset = dataset.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

dataset

<MapDataset shapes: ((256, None), (256, None)), types: (tf.int16, tf.int16)>

In [5]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return tf.keras.Sequential([
        tf.keras.layers.Masking(mask_value=0, batch_input_shape=[batch_size, None]),
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size),
    ])

In [6]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def average_batch_length(true_labels, predictions):
    return tf.shape(true_labels)[1]

checkpoint_dir = './training_checkpoints-1' # Directory where the checkpoints will be saved
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") # Name of the checkpoint files

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

class ModelStateResetter(tf.keras.callbacks.Callback):
    def __init__(self):
        self.last_total_length = 0

    def on_batch_end(self, batch, logs={}):
        average_batch_length = logs.get('average_batch_length', 0)
        total_length = int(round(average_batch_length * (batch + 1)))
        current_batch_length = total_length - self.last_total_length
        self.last_total_length = total_length
        
        if current_batch_length < BATCHED_ITEM_LENGTH:
            self.model.reset_states()
        
model_state_resetter_callback = ModelStateResetter()

In [8]:
total_epochs = 3

model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=512, rnn_units=1024, batch_size=BATCH_SIZE)
model.compile(optimizer='adam', loss=loss, metrics=[average_batch_length])

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/3
3916/3916 [==============================] - 5079s 1s/step - loss: 0.5771 - average_batch_length: 254.7786
Epoch 2/3
3916/3916 [==============================] - 4956s 1s/step - loss: 0.4256 - average_batch_length: 254.6719
Epoch 3/3
3945/3945 [==============================] - 4996s 1s/step - loss: 0.3773 - average_batch_length: 254.7265


In [9]:
total_epochs = 3

model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=512, rnn_units=1024, batch_size=BATCH_SIZE)
model.compile(optimizer='adamax', loss=loss, metrics=[average_batch_length])

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/3
4036/4036 [==============================] - 5064s 1s/step - loss: 0.6517 - average_batch_length: 254.7919
Epoch 2/3
3947/3947 [==============================] - 5070s 1s/step - loss: 0.4908 - average_batch_length: 254.7727
Epoch 3/3
4092/4092 [==============================] - 5230s 1s/step - loss: 0.4056 - average_batch_length: 254.7754


In [10]:
total_epochs = 3

model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=512, rnn_units=1024, batch_size=BATCH_SIZE)
model.compile(optimizer='nadam', loss=loss, metrics=[average_batch_length])

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/3
      5/Unknown - 48s 10s/step - loss: 6.8095 - average_batch_length: 256.0000

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[65536,4048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_3/dense_3/Tensordot/MatMul (defined at <ipython-input-10-50459effa591>:8) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Reshape_10/_60]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[65536,4048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_3/dense_3/Tensordot/MatMul (defined at <ipython-input-10-50459effa591>:8) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upo

In [12]:
def articles_generator():
    for index, article in enumerate(itertools.islice(articles(), 0, 1000)):
        yield np.array(subword_text_encoder.encode(article + b'\0'), dtype=TYPE)

    # Pad the article count to the batch size
    # We do this to ensure that no data is dropped
    index += 1
    while index % BATCH_SIZE != 0:
        yield np.array([0], dtype=TYPE)
        index += 1

In [14]:
total_epochs = 10

model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=128, rnn_units=256, batch_size=BATCH_SIZE)
model.compile(optimizer='adam', loss=loss, metrics=[average_batch_length])

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/10
420/420 [==============================] - 96s 230ms/step - loss: 1.0077 - average_batch_length: 255.6024
Epoch 2/10
449/449 [==============================] - 101s 225ms/step - loss: 0.6243 - average_batch_length: 255.3163
Epoch 3/10
422/422 [==============================] - 96s 227ms/step - loss: 0.6232 - average_batch_length: 255.1374
Epoch 4/10
422/422 [==============================] - 97s 230ms/step - loss: 0.5804 - average_batch_length: 254.9882
Epoch 5/10
411/411 [==============================] - 94s 228ms/step - loss: 0.5590 - average_batch_length: 254.9197
Epoch 6/10
459/459 [==============================] - 103s 225ms/step - loss: 0.4714 - average_batch_length: 255.6514
Epoch 7/10
444/444 [==============================] - 100s 225ms/step - loss: 0.4646 - average_batch_length: 255.1937
Epoch 8/10
438/438 [==============================] - 97s 222ms/step - loss: 0.4533 - average_batch_length: 255.2603
Epoch 9/10
423/423 [==============================] - 94s 222

In [15]:
total_epochs = 10

model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=128, rnn_units=256, batch_size=BATCH_SIZE)
model.compile(optimizer='adamax', loss=loss, metrics=[average_batch_length])

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/10
450/450 [==============================] - 100s 221ms/step - loss: 1.0525 - average_batch_length: 255.6067
Epoch 2/10
444/444 [==============================] - 98s 220ms/step - loss: 0.9564 - average_batch_length: 255.2050
Epoch 3/10
459/459 [==============================] - 102s 222ms/step - loss: 0.7849 - average_batch_length: 255.6514
Epoch 4/10
459/459 [==============================] - 103s 223ms/step - loss: 0.6610 - average_batch_length: 255.6514
Epoch 5/10
425/425 [==============================] - 97s 227ms/step - loss: 0.6641 - average_batch_length: 255.4941
Epoch 6/10
415/415 [==============================] - 94s 227ms/step - loss: 0.6778 - average_batch_length: 255.1277
Epoch 7/10
459/459 [==============================] - 104s 226ms/step - loss: 0.6177 - average_batch_length: 255.6514
Epoch 8/10
427/427 [==============================] - 96s 225ms/step - loss: 0.6588 - average_batch_length: 255.2553
Epoch 9/10
459/459 [==============================] - 104s 2

In [16]:
total_epochs = 10

model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=128, rnn_units=256, batch_size=BATCH_SIZE)
model.compile(optimizer='nadam', loss=loss, metrics=[average_batch_length])

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/10
401/401 [==============================] - 97s 241ms/step - loss: 0.9794 - average_batch_length: 254.8578
Epoch 2/10
386/386 [==============================] - 92s 239ms/step - loss: 0.7209 - average_batch_length: 254.8964
Epoch 3/10
459/459 [==============================] - 105s 229ms/step - loss: 0.5681 - average_batch_length: 255.6514
Epoch 4/10
413/413 [==============================] - 94s 227ms/step - loss: 0.5930 - average_batch_length: 255.3390
Epoch 5/10
398/398 [==============================] - 92s 232ms/step - loss: 0.5938 - average_batch_length: 254.8216
Epoch 6/10
418/418 [==============================] - 99s 237ms/step - loss: 0.5357 - average_batch_length: 255.1794
Epoch 7/10
433/433 [==============================] - 101s 233ms/step - loss: 0.4959 - average_batch_length: 255.1455
Epoch 8/10
424/424 [==============================] - 98s 232ms/step - loss: 0.4870 - average_batch_length: 255.3679
Epoch 9/10
422/422 [==============================] - 102s 243

In [17]:
total_epochs = 10

model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=128, rnn_units=256, batch_size=BATCH_SIZE)
model.compile(optimizer='adadelta', loss=loss, metrics=[average_batch_length])

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/10
447/447 [==============================] - 100s 223ms/step - loss: 8.2833 - average_batch_length: 255.3020
Epoch 2/10
458/458 [==============================] - 100s 218ms/step - loss: 8.2221 - average_batch_length: 255.3559
Epoch 3/10
459/459 [==============================] - 101s 220ms/step - loss: 7.2853 - average_batch_length: 255.6405
Epoch 4/10
436/436 [==============================] - 97s 223ms/step - loss: 2.2640 - average_batch_length: 255.4702
Epoch 5/10
425/425 [==============================] - 96s 225ms/step - loss: 1.0730 - average_batch_length: 255.4941
Epoch 6/10
417/417 [==============================] - 94s 226ms/step - loss: 1.0878 - average_batch_length: 255.4245
Epoch 7/10
459/459 [==============================] - 104s 226ms/step - loss: 0.9963 - average_batch_length: 255.6514
Epoch 8/10
433/433 [==============================] - 97s 225ms/step - loss: 1.0504 - average_batch_length: 255.1455
Epoch 9/10
459/459 [==============================] - 103s 2

In [18]:
total_epochs = 10

model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=128, rnn_units=256, batch_size=BATCH_SIZE)
model.compile(optimizer='adagrad', loss=loss, metrics=[average_batch_length])

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/10
445/445 [==============================] - 100s 224ms/step - loss: 0.9707 - average_batch_length: 255.4989
Epoch 2/10
444/444 [==============================] - 98s 221ms/step - loss: 0.6699 - average_batch_length: 255.1937
Epoch 3/10
436/436 [==============================] - 97s 222ms/step - loss: 0.6590 - average_batch_length: 255.4702
Epoch 4/10
459/459 [==============================] - 102s 222ms/step - loss: 0.6199 - average_batch_length: 255.6514
Epoch 5/10
459/459 [==============================] - 103s 225ms/step - loss: 0.6164 - average_batch_length: 255.6514
Epoch 6/10
458/458 [==============================] - 101s 221ms/step - loss: 0.6157 - average_batch_length: 255.3559
Epoch 7/10
449/449 [==============================] - 101s 225ms/step - loss: 0.6264 - average_batch_length: 255.3163
Epoch 8/10
459/459 [==============================] - 106s 230ms/step - loss: 0.6114 - average_batch_length: 255.6405
Epoch 9/10
423/423 [==============================] - 99s 